In [5]:

import xgboost as xgb
import os
# os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/'
# os.environ["PATH"] += os.pathsep + r'C:\Program Files\Graphviz 2.44.1\bin'



ModuleNotFoundError: No module named 'xgboost'

In [ ]:
# cells, gene_names, patients_information = extract_data_from_pickle()
# cells, gene_names = filter_genes_by_variance(cells, gene_names)
# dataset = RNAseq_Dataset(cells, patients_information, gene_names)




In [2]:

data = np.random.rand(10, 10)  # 5 entities, each contains 10 features
label = np.random.randint(2, size=10)  # binary target
dtrain = xgb.DMatrix(data, label=label)


param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'

# evallist = [(dtest, 'eval'), (dtrain, 'train')]
evallist = [(dtrain, 'train')]


num_round = 10
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10)


# bst.save_model('0001.model')


# Prediction
#A model that has been trained or loaded can perform predictions on data sets.




NameError: name 'np' is not defined